In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util
from itertools import combinations
import numpy as np
import json
from tqdm import tqdm

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

In [ ]:
from tqdm import tqdm

In [ ]:
dataset = load_dataset("BeIR/nfcorpus", "corpus")

In [ ]:
dataset

In [ ]:
dataset_q = load_dataset("BeIR/nfcorpus", "queries")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/14.0k [00:00<?, ?B/s]

nfcorpus.py:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

queries/queries/0000.parquet:   0%|          | 0.00/84.8k [00:00<?, ?B/s]

Generating queries split:   0%|          | 0/3237 [00:00<?, ? examples/s]

In [ ]:
dataset_q

DatasetDict({
    queries: Dataset({
        features: ['_id', 'title', 'text'],
        num_rows: 3237
    })
})

In [ ]:
dataset_qrel = load_dataset("BeIR/nfcorpus-qrels", split="test")

README.md:   0%|          | 0.00/14.0k [00:00<?, ?B/s]

train.tsv:   0%|          | 0.00/2.50M [00:00<?, ?B/s]

dev.tsv:   0%|          | 0.00/258k [00:00<?, ?B/s]

test.tsv:   0%|          | 0.00/280k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/110575 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11385 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12334 [00:00<?, ? examples/s]

In [ ]:
dataset_qrel

Dataset({
    features: ['query-id', 'corpus-id', 'score'],
    num_rows: 12334
})

In [ ]:
from collections import defaultdict

# Initialize a defaultdict to store query-doc pairs
qrels = defaultdict(dict)

# Iterate over the dataset and populate the dictionary
for qrel in dataset_qrel:
    qrels[qrel["query-id"]][qrel["corpus-id"]] = qrel["score"]

# Convert defaultdict back to a regular dictionary if needed
qrels = dict(qrels)

In [ ]:
qrels

{'PLAIN-2': {'MED-2427': 2,
  'MED-10': 2,
  'MED-2429': 2,
  'MED-2430': 2,
  'MED-2431': 2,
  'MED-14': 2,
  'MED-2432': 2,
  'MED-2428': 1,
  'MED-2440': 1,
  'MED-2434': 1,
  'MED-2435': 1,
  'MED-2436': 1,
  'MED-2437': 1,
  'MED-2438': 1,
  'MED-2439': 1,
  'MED-3597': 1,
  'MED-3598': 1,
  'MED-3599': 1,
  'MED-4556': 1,
  'MED-4559': 1,
  'MED-4560': 1,
  'MED-4828': 1,
  'MED-4829': 1,
  'MED-4830': 1},
 'PLAIN-12': {'MED-2513': 2,
  'MED-5237': 2,
  'MED-2517': 2,
  'MED-2518': 2,
  'MED-2519': 2,
  'MED-2520': 2,
  'MED-2521': 2,
  'MED-2514': 1,
  'MED-2943': 1,
  'MED-5322': 1,
  'MED-5323': 1,
  'MED-5324': 1,
  'MED-5325': 1,
  'MED-5326': 1,
  'MED-5327': 1,
  'MED-5328': 1,
  'MED-5329': 1,
  'MED-5330': 1,
  'MED-5331': 1,
  'MED-5332': 1,
  'MED-5333': 1,
  'MED-5334': 1,
  'MED-5335': 1,
  'MED-5363': 1,
  'MED-5337': 1,
  'MED-5338': 1,
  'MED-5339': 1,
  'MED-5340': 1,
  'MED-5341': 1,
  'MED-5342': 1},
 'PLAIN-23': {'MED-2644': 2,
  'MED-2646': 2,
  'MED-2651': 2

In [ ]:
len(qrels)

323

In [ ]:
print(f"Number of query-document pairs: {len(qrels)}")

Number of query-document pairs: 323


In [ ]:
queries = {query["_id"]: query["text"] for query in dataset_q["queries"]}

In [ ]:
print(f"Number of queries: {len(queries)}")


Number of queries: 3237


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

file_path = "/content/drive/MyDrive/646Project/sharedfiles/document_generated_queries_withdiversequeries_flan-t5-large_nfcorpus.json"

with open(file_path, "r") as file:
  data = json.load(file)


In [ ]:
document_pseudo_queries = data

In [ ]:
import random

In [ ]:
# Example dictionaries
# dictionary_1 = qrels
# dictionary_2 = document_pseudo_queries
# dictionary_3 = queries

# dictionary_1 = {
#     "query_id_1": {"doc_id_1": 0.8, "doc_id_2": 0.6},
#     "query_id_2": {"doc_id_3": 0.9, "doc_id_4": 0.7},
# }

# dictionary_2 = {
#     "doc_id_1": ["query_1", "query_2"],
#     "doc_id_2": ["query_3", "query_4", "query_5"],
#     "doc_id_3": ["query_6"],
#     "doc_id_4": ["query_7", "query_8"],
# }

# dictionary_3 = {
#     "query_id_1": "input_query_1",
#     "query_id_2": "input_query_2",
# }

# Step 1: Create a mapping from input query to related queries
input_output_mapping = {}

for query_id, doc_scores in qrels.items():
    input_query = queries[query_id]
    related_queries = set()  # Use a set to avoid duplicate queries

    # For each document associated with the query_id, find related queries
    for doc_id in doc_scores.keys():
        if doc_id in document_pseudo_queries:
            related_queries.update(document_pseudo_queries[doc_id])  # Add all queries for this doc_id

    if len(related_queries) > 20:
        related_queries = set(random.sample(list(related_queries), 20))

    # Step 2: Combine related queries into a single comprehensive query
    # For simplicity, let's combine the related queries into one text, which could be enhanced further
    comprehensive_query = " ".join(list(related_queries))  # Combine queries into a single string


    # Store the input-output pair
    input_output_mapping[input_query] = comprehensive_query

# Step 3: Print the mapping (training data pairs)
# for input_query, comprehensive_query in input_output_mapping.items():
#     print(f"Input Query: {input_query}")
#     print(f"Output Query: {comprehensive_query}")


In [ ]:
for input_query, comprehensive_query in input_output_mapping.items():
    print(f"Input Query: {input_query}")
    print(f"Output Query: {comprehensive_query}")
    break

Input Query: Do Cholesterol Statin Drugs Cause Breast Cancer?
Output Query: What are some of the benefits of a plant-based diet? What is the main topic of this review? Which activity has a higher relative risk of heart disease? Does methyl- cyclodextrin-derived cholesterol apopto What is the relationship between diet and breast cancer? What are two factors that can cause the amount of plaque to increase? Does statin use after breast cancer diagnosis reduce mortality due to breast cancer and all causes? Are there any evidence of publication bias and heterogeneity in the meta-analysis? What has been found that diet can affect a person's risk for cancer? What is the effect of the phytosterol beta-sitosterol and the anti Does meta-analysis of all observational studies involving 2.4 million subjects support the hypothesis that Which women's groups have been criticized? What are the two main mechanisms of action of beta-sitosterol and t How can phytochemicals and dietary compounds be used to

In [ ]:
data = [{"input_query": input_query, "comprehensive_query": comprehensive_query}
        for input_query, comprehensive_query in input_output_mapping.items()]

In [ ]:
from datasets import Dataset

In [ ]:
len_input_output_mapping= len(data)
eval_data = data[len_input_output_mapping-10:len_input_output_mapping]
train_data = data[100:len_input_output_mapping-10]

# Convert the data into Hugging Face Dataset format
train_dataset = Dataset.from_list(train_data)
eval_dataset = Dataset.from_list(eval_data)

In [ ]:
print("Train dataset:")
print(train_dataset[:2])  # Show first 2 examples
print("Eval dataset:")
print(eval_dataset[:2])

Train dataset:
{'input_query': ['energy drinks', 'ergothioneine'], 'comprehensive_query': ["Do consumption of eggs and skin poultry increase the risk in men with prostate cancer : a What are the sources of betaine and choline in a man who has advanced prostate Does post-diagnostic consumption of egg increase risk of a lethal form of prostate Are choline and vitamin B2 associated with prostate cancer risk in Swedish males? How does gut bacteria regulate metabolites? What are the symptoms of the condition 'Trimethylaminuria'? Does 11C-Choline PET/CT detect bone lesions in patients with multiple my What were the results of the analysis? Does diet after diagnosis of prostate cancer predict progression or recurrence of prostate cancer? How much red meat and poultry did they find were associated with the risk of progression of prostate cancer What are the main conclusions of the study? What are the main conclusions of this study? What are the most common causes of death in males with advance

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

# Load pre-trained model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")






tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Prepare dataset for training
def tokenize_function(batch):
    inputs = tokenizer(batch['input_query'], padding=True, truncation=True, max_length=512)
    outputs = tokenizer(batch['comprehensive_query'], padding=True, truncation=True, max_length=512)
    inputs['labels'] = outputs['input_ids']
    return inputs

In [ ]:
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/213 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
# Assuming tokenized_datasets is prepared
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=10,
    save_total_limit=2,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
)

<ipython-input-29-55974d6f33b0>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss


TrainOutput(global_step=81, training_loss=6.642069498697917, metrics={'train_runtime': 123.6953, 'train_samples_per_second': 5.166, 'train_steps_per_second': 0.655, 'total_flos': 3378258247680.0, 'train_loss': 6.642069498697917, 'epoch': 3.0})

In [ ]:
model_save_path = "/content/drive/MyDrive/646Project/sharedfiles/t5_small_finetuned_test_nfcorpus_3/"

# Save model and tokenizer
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model and tokenizer saved to {model_save_path}")

Model and tokenizer saved to /content/drive/MyDrive/646Project/sharedfiles/t5_small_finetuned_test_nfcorpus_3/


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the fine-tuned model and tokenizer
new_model = T5ForConditionalGeneration.from_pretrained(model_save_path)
new_tokenizer = T5Tokenizer.from_pretrained(model_save_path)  # Use the correct tokenizer

In [ ]:
test_data = data[:100]


In [ ]:
test_data[0]

{'input_query': 'Do Cholesterol Statin Drugs Cause Breast Cancer?',
 'comprehensive_query': "What are some of the benefits of a plant-based diet? What is the main topic of this review? Which activity has a higher relative risk of heart disease? Does methyl- cyclodextrin-derived cholesterol apopto What is the relationship between diet and breast cancer? What are two factors that can cause the amount of plaque to increase? Does statin use after breast cancer diagnosis reduce mortality due to breast cancer and all causes? Are there any evidence of publication bias and heterogeneity in the meta-analysis? What has been found that diet can affect a person's risk for cancer? What is the effect of the phytosterol beta-sitosterol and the anti Does meta-analysis of all observational studies involving 2.4 million subjects support the hypothesis that Which women's groups have been criticized? What are the two main mechanisms of action of beta-sitosterol and t How can phytochemicals and dietary com

In [ ]:
# test_data_refined = [{"input_query": input["input_query"] + "?"}
for input in test_data:
      print(input['input_query'])
      break


Do Cholesterol Statin Drugs Cause Breast Cancer?


In [ ]:
test_data_refined = [{"input_query": input['input_query']} for input in test_data]  # Add '?' to each query to make it a complete question


In [ ]:
test_data_refined

[{'input_query': 'Do Cholesterol Statin Drugs Cause Breast Cancer?'},
 {'input_query': 'Exploiting Autophagy to Live Longer'},
 {'input_query': 'How to Reduce Exposure to Alkylphenols Through Your Diet'},
 {'input_query': 'What’s Driving America’s Obesity Problem?'},
 {'input_query': 'Who Should be Careful About Curcumin?'},
 {'input_query': 'Foods for Glaucoma'},
 {'input_query': 'What is Actually in Chicken Nuggets?'},
 {'input_query': 'What Do Meat Purge and Cola Have in Common?'},
 {'input_query': 'Chronic Headaches and Pork Parasites'},
 {'input_query': 'Stopping Heart Disease in Childhood'},
 {'input_query': 'Food Dyes and ADHD'},
 {'input_query': 'How Citrus Might Help Keep Your Hands Warm'},
 {'input_query': 'Starving Tumors of Their Blood Supply'},
 {'input_query': 'Are Dental X-Rays Safe?'},
 {'input_query': 'How Should I Take Probiotics?'},
 {'input_query': 'Breast Cancer & Alcohol: How Much is Safe?'},
 {'input_query': 'Diet and Cellulite'},
 {'input_query': 'Best Treatment

In [ ]:
from tqdm import tqdm

In [ ]:


# Example test dataset (a list of input queries)
# test_data = [
#     {"input_query": "Does statin use reduce breast cancer mortality?"},
#     {"input_query": "How does methylmercury affect the nervous system?"},
#     {"input_query": "Does inositol hexaphosphate modulate neutrophils?"}
# ]

# Function to predict the comprehensive query
def predict_comprehensive_query(test_data_refined):

    # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # new_model.to(device)
    # tokenizer.to(device)

    predictions = []
    # test_data_refined_short = list(test_data_refined)[:2]
    for item in tqdm(test_data_refined):
        input_query = item["input_query"]

        # Tokenize the input query
        inputs = new_tokenizer(input_query, return_tensors="pt", padding=True, truncation=True, max_length=512)

        # Generate predictions using the model
        # with torch.no_grad():  # Disable gradient calculation for inference
        output_ids = new_model.generate(inputs["input_ids"], max_length=512, num_beams=4, early_stopping=True)

        # Decode the generated token IDs back to text
        predicted_query = new_tokenizer.decode(output_ids[0], skip_special_tokens=True)
        predictions.append(predicted_query)
        # break

    return predictions




In [ ]:
# Get predictions for the test data
predictions = predict_comprehensive_query(test_data_refined)

100%|██████████| 100/100 [05:42<00:00,  3.42s/it]


In [ ]:
queries_dict = list(qrels.items())[:100]

In [ ]:
# Model names
generative_model_name = "google/flan-t5-large"
similarity_model_name = "all-mpnet-base-v2"
chunk_size=500
overlap=0
threshold=0.8

In [ ]:
similarity_model = SentenceTransformer(similarity_model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def calculate_similarity_score(user_pseudo_query, document_pseudo_queries, similarity_model):
    """
    Calculate the similarity score for a query-document pair.

    Parameters:
    - query: The user query (string).
    - document_pseudo_queries: List of pseudo-queries for the document.
    - similarity_model: Preloaded Sentence Transformer model.

    Returns:
    - max_similarity: Maximum similarity score between query and pseudo-queries.
    """
    # Compute embeddings for the query and pseudo-queries
    # query_embedding = similarity_model.encode(user_pseudo_query, convert_to_tensor=True)
    # # pseudo_query_embeddings = similarity_model.encode(document_pseudo_queries, convert_to_tensor=True)

    # # Compute cosine similarity scores
    # similarity_scores = util.cos_sim(query_embedding, document_pseudo_queries)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    similarity_model_gpu = similarity_model.to(device)

    # Compute embedding for the query on the specified device
    query_embedding = similarity_model_gpu.encode(
        user_pseudo_query,
        convert_to_tensor=True,
        device=device
    )

    if query_embedding.dim() == 1:
        query_embedding = query_embedding.unsqueeze(0)

    # Ensure pseudo-query embeddings are also on the same device
    document_pseudo_queries = document_pseudo_queries.to(device)
    # print(query_embedding.shape)
    # print(document_pseudo_queries.shape)

    # Compute cosine similarity scores
    similarity_scores = util.cos_sim(query_embedding, document_pseudo_queries)

    # Find the maximum similarity score
    max_similarity = torch.max(similarity_scores).item()
    return max_similarity



In [ ]:
def rank_documents_by_query(query, document_pseudo_queries_embeddings, similarity_model):
    """
    Rank documents based on the similarity of their pseudo-queries to the query.

    Parameters:
    - query: The user query (string).
    - document_queries_dict: Dictionary where keys are document IDs and values are lists of pseudo-queries.
    - similarity_model: Preloaded Sentence Transformer model.

    Returns:
    - ranked_scores: List of tuples (document_id, score) sorted by descending scores.
    """
    document_scores = {}
    for doc_id, pseudo_queries_embeddings in document_pseudo_queries_embeddings.items():
        # print(doc_id)
        # print(pseudo_queries)
        # print("yoo")
        score = calculate_similarity_score(query, pseudo_queries_embeddings, similarity_model)
        document_scores[doc_id] = score

    # Sort documents by scores in descending order
    ranked_scores = sorted(document_scores.items(), key=lambda x: x[1], reverse=True)
    return ranked_scores

In [ ]:
# queries_dict = list(qrels.items())[:2]
test_dict = {}
document_pseudo_queries_embeddings={}
device = "cuda" if torch.cuda.is_available() else "cpu"
similarity_model_gpu = similarity_model.to(device)

for doc_id, pseudo_queries in tqdm(document_pseudo_queries.items()):
  # print(pseudo_queries)
  # print(len(pseudo_queries))
  pseudo_query_embeddings = similarity_model_gpu.encode(pseudo_queries, convert_to_tensor=True, device = device)
  document_pseudo_queries_embeddings[doc_id] = pseudo_query_embeddings
  # # print(doc_id)
  # print(pseudo_query_embeddings[0])
  # print(len(pseudo_query_embeddings))
  # # print(document_pseudo_queries_embeddings)
  # break



100%|██████████| 3633/3633 [01:01<00:00, 59.44it/s]


In [ ]:
qrel_pseudo ={}
index=0
for query_id, doc_score in tqdm(queries_dict):

  user_pseudo_query =predictions[index]
  qrel_pseudo[str(query_id)]=user_pseudo_query
  # user_pseudo_query = queries[str(query_id)]
  doc_rank_for_query = rank_documents_by_query(user_pseudo_query, document_pseudo_queries_embeddings, similarity_model)
  # print(type(doc_rank_for_query))
  test_dict[query_id]={}
  index+=1
  for doc_id, score in doc_rank_for_query:
    test_dict[query_id][doc_id] = float(score)


100%|██████████| 100/100 [1:38:49<00:00, 59.30s/it]


In [ ]:
!pip install pytrec_eval

  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp310-cp310-linux_x86_64.whl size=308210 sha256=f601fb9d6458e404a89adcb0ece7a83916939fe2613f9ed3afd1cf90caeb7aa6
  Stored in directory: /root/.cache/pip/wheels/51/3a/cd/dcc1ddfc763987d5cb237165d8ac249aa98a23ab90f67317a8
Successfully built pytrec_eval


In [ ]:
import pytrec_eval

In [ ]:
top_n = 10
top_documents = {}
for query_id, doc_scores in test_dict.items():
  # Directly take the top N documents (already sorted)
  top_documents[query_id] = dict(list(doc_scores.items())[:top_n])

In [ ]:
evaluator = pytrec_eval.RelevanceEvaluator(qrels, {'P.3,5,10', 'recall.3,5,10', 'ndcg_cut.3,5,10', 'map_cut.3,5,10'})
result = evaluator.evaluate(top_documents)
print(result)
metrics = ['P','ndcg_cut', 'recall', 'map_cut']
cutoffs = [3,5,10]
scores = {f'{metric}_{cutoff}': 0 for metric in metrics for cutoff in cutoffs}
for key in result:
  for metric in metrics:
    for cutoff in cutoffs:
      scores[f'{metric}_{cutoff}'] += result[key][f'{metric}_{cutoff}']
run_length = len(test_dict)
for score in scores:
  scores[score] /= run_length

{'PLAIN-2': {'P_3': 1.0, 'P_5': 0.8, 'P_10': 0.5, 'recall_3': 0.125, 'recall_5': 0.16666666666666666, 'recall_10': 0.20833333333333334, 'ndcg_cut_3': 0.8826803184943108, 'ndcg_cut_5': 0.7840048711692542, 'ndcg_cut_10': 0.6018709432457179, 'map_cut_3': 0.125, 'map_cut_5': 0.16666666666666666, 'map_cut_10': 0.1898148148148148}, 'PLAIN-12': {'P_3': 0.0, 'P_5': 0.2, 'P_10': 0.1, 'recall_3': 0.0, 'recall_5': 0.03333333333333333, 'recall_10': 0.03333333333333333, 'ndcg_cut_3': 0.0, 'ndcg_cut_5': 0.13120507751234178, 'ndcg_cut_10': 0.09456701548023277, 'map_cut_3': 0.0, 'map_cut_5': 0.006666666666666667, 'map_cut_10': 0.006666666666666667}, 'PLAIN-23': {'P_3': 0.3333333333333333, 'P_5': 0.4, 'P_10': 0.2, 'recall_3': 0.011111111111111112, 'recall_5': 0.022222222222222223, 'recall_10': 0.022222222222222223, 'ndcg_cut_3': 0.23463936301137822, 'ndcg_cut_5': 0.30078518014914984, 'ndcg_cut_10': 0.19518900079066107, 'map_cut_3': 0.011111111111111112, 'map_cut_5': 0.015555555555555555, 'map_cut_10': 

In [ ]:
#new scores
scores

{'P_3': 0.2533333333333334,
 'P_5': 0.21999999999999992,
 'P_10': 0.15899999999999995,
 'ndcg_cut_3': 0.2595646098780326,
 'ndcg_cut_5': 0.24476690597021314,
 'ndcg_cut_10': 0.21903204551616576,
 'recall_3': 0.06205631984584467,
 'recall_5': 0.07955932973791319,
 'recall_10': 0.11013072786052273,
 'map_cut_3': 0.05308475676353054,
 'map_cut_5': 0.06423314807838028,
 'map_cut_10': 0.07676395382364268}

In [ ]:
#old scores
scores

{'P_3': 0.26,
 'P_5': 0.22799999999999992,
 'P_10': 0.15899999999999995,
 'ndcg_cut_3': 0.273141003422413,
 'ndcg_cut_5': 0.25989030638497485,
 'ndcg_cut_10': 0.22414160223150736,
 'recall_3': 0.06843056629997615,
 'recall_5': 0.09189320874728418,
 'recall_10': 0.10517061801199526,
 'map_cut_3': 0.05662811573318425,
 'map_cut_5': 0.06922220571637283,
 'map_cut_10': 0.07850247032193344}

In [ ]:
import json
with open("/content/drive/MyDrive/646Project/sharedfiles/qrel_pseudo_t5smallpart2finetuned_nfcorpus", "w") as json_file:
  json.dump(qrel_pseudo, json_file, indent=4)

In [ ]:
test_data[0]['input_query']

'Do Cholesterol Statin Drugs Cause Breast Cancer?'

In [ ]:
test_data[1]['input_query']

'Exploiting Autophagy to Live Longer'

In [ ]:
predictions

['Die Medikamente mit Cholesterol-Statinen tragen zu Brustkrebs bei?',
 'Exploiting Autophagy to Live Longer Live, Exploiting Autophagy, Exploiting Autophagy, Exploiting Autophagy to Live Longer Live.']

In [ ]:
# test_data

In [ ]:
# Print the predicted comprehensive queries
for input_query, predicted_query in zip([item["input_query"] for item in test_data], predictions):
    print(f"Input Query: {input_query}")
    print(f"Predicted Comprehensive Query: {predicted_query}")
    print("-" * 50)